# FH Aachen - Fachbereich Maschinenbau & Mechatronik

## Abschlussprojekt WS 2022 / 2023

### im Modul: 81307 - Datenmanagement & Leittechnik

#### bei Prof. Dr. Ing. Stephan Kallweit

#### am 07.03.2023

vorgelegt von:

##### Lara Marie May
##### Matrikelnummer: 3138207 
##### Mail: lara-marie.may@alumni.fh-aachen.de

und

##### Sonja Sternberg
##### Matrikelnummer: 3141153
##### Mail: sonja.sternberg@alumni.fh-aachen.de

_______________________________________________________________________

### Hinweise zur Verwendung der Programme

Das vorliegende Projekt besteht aus verschiedenen Code-Elementen. Manche Code-Elemente müssen auf einem Mikrocontroller gespeichert werden, andere müssen in einem Jupyter-Notebook ausgeführt werden. Wofür die einzelnen Code-Elemente benötigt werden und wo sie ausgeführt werden, wird im Einzelnen untenstehend erläutert. In jedem Fall die ist Reihenfolge der Ausführung nicht von Bedeutung. Der Code ist so geschrieben, dass das Programm auf benötigte Daten wartet, ohne eine Fehlermeldung zu erzeugen.

### Einleitung

Im Modul Datenmanagement und Leittechnik haben die Studierenden die Aufgabe, im Rahmen eines Projektes ein Programm zu schreiben, das auf der Programmiersprache Python basiert. Das Programm soll verschiedene Betriebszustände klassifizieren und damit die wesentlichen Aspekte der Prozessüberwachung abbilden. Ziel ist es, die in der Lehrveranstaltung erworbenen Kenntnisse der Prozesserfassung und -überwachung in der Praxis anzuwenden, um eine gegebene Problemstellung zu lösen. Im Mittelpunkt der diesjährigen Problemstellung steht eine Waschmaschine. Der geschriebene Code soll mindestens vier verschiedene Betriebszustände einer Waschmaschine klassifizieren können. Die erfassten Daten sollen auch außerhalb des Heim-Netzwerkes abgerufen werden können. Zudem sollen die Daten über ein grafisches User-Interface visualisiert werden.

Die Datenerfassung erfolgt über ein zuvor ausgeteiltes IoT-Kit. Darauf angeschlossen ist ein MPU 6050 Beschleunigungssensor, der die Vibrationen misst. Das Projekt wird an einer Waschmaschine von Bauknecht durchgeführt. Die Waschmaschine ist unter der Küchen-Arbeitsplatte eingeklemmt, sodass der Bewegungsspielraum der Maschine eingeschränkt ist und die Vibrationen somit leicht gedämpft werden.

Das betrachtete Waschprogramm hat eine Länge von 2 Stunden und 49 Minuten und beinhaltet die folgenden Vorgänge:
* Wasser einspülen
* Vorwäsche
* Waschen
* Schleudern
* Stillstand

Diese fünf Zustände sollen durch maschinelles Lernen erkannt werden.

### Programme auf dem Mikrocontroller

Für die Erfassung der Vibrationsdaten muss zuerst der Mikrocontroller programmiert werden. Hierfür werden verschiedene Codes auf den Mikrocontroller gespielt. Die Datei 'boot.py' wird beim Start / beim Reset des Mikrocontrollers zuerst ausgeführt und lädt das eigentliche Programm in den Speicher. Die Dateien 'imu.py' und 'vector3d.py' machen XY. Sie stellen Hilfsdateien dar, die der Lagebestimmung dienen und wurden vorab zur Verfügung gestellt. Der eigentlich Programmcode befindet sich in 'main.py'.

Der Programmcode gibt zu Beginn eine Liste aller aktuell auf auf dem Mikrocontroller geflashten Dateien aus. So kann eingesehen werden, ob alle benötigten Dateien zur Verfügung stehen. Anschließend stellt der Mikrocontroller eine Verbindung zum WLAN her und zieht die aktuelle Zeit aus dem Netzwerk. Im nächsten Schritt wird der I2C-Bus initialisiert, der den Beschleunigungssensor ansprechen soll. Mit der Initialisierung der Variablen und Funktionen, wird auch das Auslesen der Beschleunigungsdaten gestartet. Die aktuellen Beschleunigungswerte werden in allen drei Achsen, sprich in X-Richtung, in Y-Richtung und in Z-Richtung erfasst. Anschließend wird eine TCP-Verbindung mit der Server-Seite hergestellt. Der Mikrocontroller stellt dabei die Client-Seite der TCP-Kommunikation dar. Wenn die Verbindung hergestellt wurde, erfolgt die Übermittlung der Beschleunigungsdaten per TCP und es wird ein Timer gestartet.  Infolgedessen wird eine Callback-Funktion aufgerufen, die die Anzahl der Durchläufe zählt. Beim Erreichen eines Wertes von 200, wird der Counter auf null zurückgesetzt. Hat der Counter den Wert null, wird per TCP ein Zeitstempel mit der aktuellen Zeit gesendet. Das Programm stoppt automatisch, wenn die TCP-Verbindung von der Server-Seite abbricht.

Der Mikrocontroller kann in einer Entwicklungsumgebung für Mikro-Python, wie zum Beispiel Thonny, programmiert werden.

#### Bestimmung der Abtastfrequenz:

Die maximale Drehzahl der Waschmaschine n_max beträgt 1600 min-1. Die maximale Drehfrequenz der Waschmaschine f_max ergibt sich aus der folgenden Formel:

f_max = (n_max [1/min])/(60 [s/min] )

Nach einsetzen von n_max kann die maximale Drehfrequenz errechnet werden: (1600 [1/min])/(60 [s/min]) = 26,67 Hz

Die maximale Drehfrequenz des Waschprogramms f_max beträgt dementsprechend 26,67 Hz. Um Aliasing zu verhindern, empfiehlt es sich mit einer Abtastrate zu arbeiten, die mindestens doppelt so hoch wie die höchste Frequenz ist. Das heißt, die Abtastfrequenz muss mindestens das doppelte der maximalen Drehfrequenz der Waschmaschine f_max, sprich 2 * 26,67 Hz = 53,34 Hz betragen. Um eine möglichst genaue Abtastung des Eingangssignals zu erreichen sowie der Einfachheit halber, wird im Projektverlauf mit einer Frequenz von 100 Hz abgetastet.

### Programme in Jupyter-Notebooks

Parallel zu dem Main-Programm laufen zwei Programme in der Entwicklungsumgebung Jupyter. Ein Programm dient dem Empfang der Beschleunigungsdaten über TCP sowie dessen Verarbeitung. Mit dem anderen Programm werden die Daten in einem grafischen User-Interface visualisiert.

#### Datenempfang und Datenverarbeitung

Mit dem ersten Programm wird die Serverseite für die TCP-Kommunikation eingerichtet, die vom Mikrocontroller gesendeten Daten empfangen, verarbeitet und schließlich in die Datenbank hochgeladen. Im ersten Schritt wird ein Socket erzeugt und darauf gewartet, dass der Mikrocontroller sich mit dem Server verbindet. Danach startet eine While-Schleife,in der die vom Mikrocontroller gesendeten Daten empfangen und entschlüsselt werden. Im nächsten Schritt werden die Daten gefiltert. Dabei wird geprüft, ob die Beschleunigungsdaten vollständig und im richtigen Format übermittelt wurden. Alle unvollständigen oder falsch dargestellten Datensätze werden hierbei gelöscht. Schließlich werden die Daten in die cloudbasierte Datenbank 'MongoDB Atlas' hochgeladen. Diese While-Schleife wird solange durchlaufen, bis das Programm manuell geschlossen wird.

#### Visualisierung im einer Benutzeroberfläche

Mit dem zweiten Programm in Jupyter sollen die aufgenommenen Daten und der ermittelte Betriebszustand in einem grafischen User-Interface visualisiert werden. Beim Start des Programms wird ein Diagramm in einem Pop-up-Fenster erzeugt. In diesem Diagramm werden die aktuellsten 2.000 Datensätze dargestellt. So wird ein schneller Überblick über die Datenlage ermöglicht. Nach dem Schließen dieses Pop-up-Fensters wechselt die Benutzeroberfläche in den nächsten Modus. Dabei werden zwei Dialogfenster aktiv. Im ersten Dialogfenster werden wichtige Informationen und Statusmeldungen des Codes angezeigt. Im zweiten Dialogfenster werden weiterführende Informationen zu den erfassten Daten dargestellt. In einer Anzeige werden die zuletzt gemessenen Beschleunigungswerte dargestellt. Zudem wird ein Diagramm ausgegeben, das die letzten 500 Datensätze visuell abbildet. Die Benutzeroberfläche bietet zudem verschiedene Möglichkeiten, die Datenausgabe anzupassen. Der Nutzer kann die Datenmenge und den Informationsgehalt somit selber bestimmen. Nach Benutzung kann das Programm über einen Stop-Button unterbrochen werden.

#### Anwendung des maschinellen Lernens

Bei der Anwendung des maschinellen Lernens werden zunächst fünf Zustände klassifiziert und trainiert:
* Wasser einspülen
* Vorwäsche
* Waschen
* Schleudern
* Stillstand

Dabei fällt auf, dass das Programm Schwierigkeiten hat, Waschen und Vorwäsche zu unterscheiden. Zusätzlich lässt sich das Spülen nicht eindeutig klassifizieren. Es ist anzunehmen, dass dieser Effekt durch die Platzierung der Waschmaschine unter der Arbeitsplatte verstärkt wird. Daher wurde entschieden, nur drei Klassen zu bilden und damit nur drei Zustände zu unterscheiden. Damit soll die Genauigkeit des Algorithmus erhöht werden.:
* Waschen
* Schleudern
* Stillstand